In [1]:
import numpy as np
import folium
from folium.plugins import HeatMap
import networkx as nx

# Room dimensions
ROWS, COLS = 10, 10

# Simulated smoke levels
np.random.seed(42)
smoke_levels = np.random.randint(0, 100, size=(ROWS, COLS)).astype(float)

# Normalize smoke levels
smoke_min = np.min(smoke_levels)
smoke_max = np.max(smoke_levels)
normalized = (smoke_levels - smoke_min) / (smoke_max - smoke_min + 1e-9)


# Coordinates mapping (for visualization only)
def grid_to_latlon(i, j):
    return 26.5 - i * 0.001, 80.2 + j * 0.001


# Build weighted graph
G = nx.grid_2d_graph(ROWS, COLS)
for i, j in G.nodes():
    G.nodes[(i, j)]["weight"] = smoke_levels[i][j]
for u, v in G.edges():
    G.edges[u, v]["weight"] = (G.nodes[u]["weight"] + G.nodes[v]["weight"]) / 2

# Find safest path
start, end = (0, 0), (ROWS - 1, COLS - 1)
path = nx.dijkstra_path(G, start, end, weight="weight")
path_coords = [grid_to_latlon(i, j) for i, j in path]

# Prepare heatmap data
heat_data = [
    [*grid_to_latlon(i, j), float(normalized[i][j])]
    for i in range(ROWS)
    for j in range(COLS)
]

# Create map
m = folium.Map(location=grid_to_latlon(ROWS // 2, COLS // 2), zoom_start=17)

# Add heatmap layer
HeatMap(heat_data).add_to(m)

# Add escape path
folium.PolyLine(path_coords, color="blue", weight=4).add_to(m)

# Render in Jupyter Notebook
m

In [5]:
import numpy as np
import folium
from folium.plugins import HeatMap, MiniMap, Fullscreen, MeasureControl
import networkx as nx
import os
import webbrowser

# Grid size
ROWS, COLS = 15, 15

# Simulate smoke levels using gamma for more contrast (low and high values)
np.random.seed(42)
smoke_levels = np.random.gamma(
    shape=1.5, scale=50, size=(ROWS, COLS)
)  # Range approx 0–300+

# Normalize for heatmap
smoke_min = np.min(smoke_levels)
smoke_max = np.max(smoke_levels)
normalized = (smoke_levels - smoke_min) / (smoke_max - smoke_min + 1e-9)


# Convert grid to lat/lon in IIT Kanpur
def grid_to_latlon(i, j):
    lat_start, lat_end = 26.5125, 26.5105
    lon_start, lon_end = 80.2305, 80.2325
    lat = lat_start + (lat_end - lat_start) * (i / (ROWS - 1))
    lon = lon_start + (lon_end - lon_start) * (j / (COLS - 1))
    return float(lat), float(lon)


# Build graph with smoke weights
G = nx.grid_2d_graph(ROWS, COLS)
for i, j in G.nodes():
    G.nodes[(i, j)]["weight"] = float(smoke_levels[i][j])
for u, v in G.edges():
    G.edges[u, v]["weight"] = (G.nodes[u]["weight"] + G.nodes[v]["weight"]) / 2

# Define start and exits
start = (0, 0)
exits = [(0, COLS - 1), (ROWS - 1, 0), (ROWS - 1, COLS - 1)]

# Find best path by total exposure (sum of weights)
min_sum_cost = float("inf")
sum_path = None
sum_exit = None

for ex in exits:
    try:
        path = nx.dijkstra_path(G, start, ex, weight="weight")
        cost = sum(G.nodes[n]["weight"] for n in path)
        if cost < min_sum_cost:
            min_sum_cost = cost
            sum_path = path
            sum_exit = ex
    except nx.NetworkXNoPath:
        continue

# Find best path by minimum peak danger (max of weights)
min_max_cost = float("inf")
max_path = None
max_exit = None

for ex in exits:
    try:
        path = nx.dijkstra_path(G, start, ex, weight="weight")
        cost = max(G.nodes[n]["weight"] for n in path)
        if cost < min_max_cost:
            min_max_cost = cost
            max_path = path
            max_exit = ex
    except nx.NetworkXNoPath:
        continue

# Create folium map
m = folium.Map(
    location=grid_to_latlon(ROWS // 2, COLS // 2), zoom_start=20, tiles="OpenStreetMap"
)

# Add heatmap
heat_data = [
    grid_to_latlon(i, j)
    for i in range(ROWS)
    for j in range(COLS)
    if normalized[i][j] > 0.2
]
HeatMap(heat_data, radius=20, blur=25, max_zoom=1).add_to(m)

# Plot total exposure path (blue dashed)
sum_coords = [grid_to_latlon(i, j) for i, j in sum_path]
for i in range(len(sum_coords) - 1):
    folium.PolyLine(
        [sum_coords[i], sum_coords[i + 1]],
        color="blue",
        weight=4,
        dash_array="6,6",
        tooltip="Path (Min Total Smoke)",
    ).add_to(m)

# Plot peak danger path (purple dotted)
max_coords = [grid_to_latlon(i, j) for i, j in max_path]
for i in range(len(max_coords) - 1):
    folium.PolyLine(
        [max_coords[i], max_coords[i + 1]],
        color="purple",
        weight=4,
        dash_array="2,8",
        tooltip="Path (Min Peak Smoke)",
    ).add_to(m)

# Start marker
folium.Marker(
    grid_to_latlon(*start),
    popup=folium.Popup("Start Point 🟢"),
    icon=folium.Icon(color="green"),
).add_to(m)

# Mark exits
for ex in exits:
    coords = grid_to_latlon(*ex)
    label = ""
    if ex == sum_exit and ex == max_exit:
        label = "Safest Exit 🚪 (Both Criteria)"
        color = "red"
    elif ex == sum_exit:
        label = "Safest Exit (Min Total)"
        color = "darkred"
    elif ex == max_exit:
        label = "Safest Exit (Min Peak)"
        color = "orange"
    else:
        label = "Alternate Exit"
        color = "lightgray"

    folium.Marker(
        coords, popup=folium.Popup(label), icon=folium.Icon(color=color)
    ).add_to(m)

# High-smoke points
for i in range(ROWS):
    for j in range(COLS):
        if normalized[i][j] > 0.6:
            folium.CircleMarker(
                location=grid_to_latlon(i, j),
                radius=3.5,
                color="black",
                fill=True,
                fill_color="crimson",
                fill_opacity=0.7,
                popup=folium.Popup(f"Smoke: {smoke_levels[i][j]:.1f}"),
            ).add_to(m)

# Controls and title
MiniMap(position="topleft").add_to(m)
Fullscreen().add_to(m)
m.add_child(MeasureControl(primary_length_unit="meters"))

# Floating title
title_html = """
     <div style="position: fixed; top: 10px; left: 50%; transform: translateX(-50%);
     background-color: white; padding: 6px 20px; border: 2px solid grey;
     border-radius: 8px; z-index:9999; font-size: 16px;">
     <b>🔥 Fire Escape Simulation (IIT Kanpur): Dual Criteria</b>
     </div>
"""
m.get_root().html.add_child(folium.Element(title_html))

# Save and open
map_path = "iitk_fire_escape_dual_criteria.html"
m.save(map_path)
webbrowser.open("file://" + os.path.realpath(map_path))


True